In [1]:
import os
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import display, Markdown

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY")

# check if API keys are set
if not OPENAI_API_KEY:
    raise ValueError("Missing OpenAI API key")
if not GEMINI_API_KEY:
    raise ValueError("Missing Gemini API key")
if not ANTHROPIC_API_KEY:
    raise ValueError("Missing Anthropic API key")

In [4]:
openai_client = OpenAI()
anthropic_client = OpenAI(api_key=ANTHROPIC_API_KEY, base_url="https://api.anthropic.com/v1")
gemini_client = OpenAI(api_key=GEMINI_API_KEY, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

In [5]:
message = "I want you to generate a customer support ticket for a 3rd party re-seller. "
message += "The ticket should be a single sentence describing a common issue a customer might face with their product or service. "
message += "Do not give any subjects, only the body of the ticket."

messages = [{"role": "user", "content": message}]

In [7]:
openai_response = openai_client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=messages
)

ticket = openai_response.choices[0].message.content
# print(f"### Generated Ticket:\n{ticket}")
display(Markdown(f"### Generated Ticket:\n{ticket}"))

### Generated Ticket:
The product I purchased is not functioning as advertised and I haven't received any response to my previous inquiries.

In [8]:
# --- Evaluator Agent: Assess ticket quality and suggest improvements ---
evaluator_prompt = (
    "You are an expert customer support evaluator. "
    "Assess the following support ticket for quality and completeness. "
    "Suggest specific improvements to make it clearer and more actionable. "
    "Ticket:\n"
    f"{ticket}\n"
    "Respond with your assessment and suggested improvements."
)

evaluator_messages = [{"role": "user", "content": evaluator_prompt}]

evaluator_response = anthropic_client.chat.completions.create(
    model="claude-3-5-haiku-latest",
    messages=evaluator_messages
)

evaluation_feedback = evaluator_response.choices[0].message.content
display(Markdown(f"**Evaluator Feedback:**\n\n{evaluation_feedback}"))

**Evaluator Feedback:**

Assessment of Support Ticket:

Current Ticket Quality: Low

Specific Issues:
1. Lack of Specific Details
- No specific product identification
- No clear description of the functional problem
- No indication of previous communication attempts

Suggested Improvements:
1. Request Additional Information:
- Product model/serial number
- Purchase date
- Specific malfunction description
- Details of previous communication attempts

2. Recommended Response Template:
"Thank you for contacting our support team. To best assist you, could you please provide:
- Complete product name and model number
- Date of purchase
- Detailed description of the issue (What exactly is not working?)
- Copies/references of previous communication
- Proof of purchase if available

Our team is committed to resolving your concern promptly."

Action Items:
- Assign ticket to appropriate product support specialist
- Set clear timeline for initial response (within 24 hours)
- Establish follow-up protocol

Quality Score: 2/10
Potential Score After Improvements: 8/10

Recommendations emphasize gathering precise information to diagnose and resolve the customer's problem efficiently and professionally.

In [9]:
# --- Optimizer Agent: Revise ticket based on evaluator feedback ---
optimizer_prompt = (
    "You are an optimizer agent. "
    "Revise the following customer support ticket based on the evaluator's feedback, "
    "aiming for clarity and actionable details.\n\n"
    f"Original Ticket:\n{ticket}\n\n"
    f"Evaluator Feedback:\n{evaluation_feedback}\n\n"
    "Provide the improved ticket only."
)

optimizer_messages = [{"role": "user", "content": optimizer_prompt}]

optimizer_response = gemini_client.chat.completions.create(
    model="gemini-2.5-flash",
    messages=optimizer_messages
)

optimized_ticket = optimizer_response.choices[0].message.content
display(Markdown(f"**Original Ticket:**\n\n{ticket}\n\n**Optimized Ticket:**\n\n{optimized_ticket}"))

**Original Ticket:**

The product I purchased is not functioning as advertised and I haven't received any response to my previous inquiries.

**Optimized Ticket:**

**Subject: Product Malfunction & Unresolved Inquiry - [Product Name/Model] - Order #[Order Number]**

Dear Support Team,

I am writing to report a significant issue with a product I purchased and to request urgent assistance, as my previous inquiries regarding this matter have not received a response.

**Product Details:**
*   **Product Name/Model:** [e.g., "ApexPro Wireless Headset, Model HP-500"]
*   **Serial Number (if applicable):** [e.g., "HP500-2023-A1B2C3D4E5"]
*   **Purchase Date:** [e.g., "November 15, 2023"]
*   **Order Number:** [e.g., "WEB-987654321"]
*   **Place of Purchase:** [e.g., "Your Company's Official Website" or "Amazon.com"]
*   **Proof of Purchase:** [e.g., "Attached is a copy of my order confirmation email."]

**Description of Malfunction:**
The [Product Name] is not functioning as advertised. Specifically:
*   **Core Issue:** The advertised noise-cancellation feature is completely non-functional. I can hear ambient sounds clearly, which defeats the primary purpose for which I purchased these headphones.
*   **Secondary Issue:** The battery life is significantly shorter than the advertised 20 hours, lasting only approximately 5-6 hours on a full charge.
*   **Expected vs. Actual:** The product was advertised as providing superior noise isolation and extended battery life for professional use, neither of which is being delivered.

**Previous Communication Attempts:**
I previously contacted your support team regarding this issue on the following dates:
*   **Date:** [e.g., "December 1, 2023"]
    *   **Method:** Email
    *   **Recipient:** [e.g., "support@yourcompany.com"]
    *   **Subject Line:** [e.g., "Issue with HP-500 Noise Cancellation - Order #987654321"]
    *   **Reference/Attachment:** [e.g., "Sent from my email address: customer@example.com, no ticket ID received."]
*   **Date:** [e.g., "December 5, 2023"]
    *   **Method:** Live Chat
    *   **Reference/Attachment:** [e.g., "Chat transcript ID: CHAT-123456789. Agent name (if known): 'Sarah K.'"]

I have not received any response or follow-up from these previous attempts. I would appreciate prompt assistance in resolving this issue. Please let me know if any further information or troubleshooting is required.

Thank you for your time.

Sincerely,

[Customer Name]
[Customer Email Address]
[Customer Phone Number (Optional)]